In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import seaborn as sns
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pickle
sns.set()

In [38]:
data = pd.read_csv('../data/final_datasets/excercise.csv')
plans = pd.read_csv('../data/final_datasets/plans.csv')
with open('../models/kmeans.pickle', 'rb') as f:
    kmeans = pickle.load(f)
with open('../models/plan_classifier.pickle', 'rb') as f:
     plan_classifier = pickle.load(f)
X_train_cols = ['level_Advanced', 'level_Beginner', 'level_Intermediate',
       'goal_ Get Fitter', 'goal_ Lose Weight', 'goal_Gain Muscle',
       'goal_Get Fitter', 'goal_Increase Endurance', 'goal_Increase Strength',
       'goal_Sports Performance', 'gender_Female', 'gender_Male',
       'gender_Male & Female']

In [39]:
def choose_plan(level, goal, gender):
    global plan_classifier
    # Convert input into a DataFrame
    input_data = pd.DataFrame({'level': [level], 'goal': [goal], 'gender': [gender]})

    # One-hot encode the input data
    input_encoded = pd.get_dummies(input_data, columns=['level', 'goal', 'gender'])

    # Ensure that input has the same columns as the model was trained on
    # This is necessary in case some categories are missing in the input
    missing_cols = set(X_train_cols) - set(input_encoded.columns)
    for col in missing_cols:
        input_encoded[col] = 0

    # Reorder columns to match the order of columns in X_train
    input_encoded = input_encoded[X_train_cols]

    # Make prediction for the given input using the trained model
    prediction = plan_classifier.predict(input_encoded)


    # Convert each string in the list to a list of strings
    daily_activities_lists = [day.split(', ') for day in prediction[0]]


    return daily_activities_lists

In [40]:
cluster_data = {}

# Iterate over each cluster label
for cluster_label in range(90):
    # Filter the dataset to get data for the current cluster
    cluster_subset = data[data["cluster"] == cluster_label]

    # Add the cluster data to the dictionary
    cluster_data[cluster_label] = cluster_subset

In [41]:
features = data[["Level", "goal", "bodyPart"]]

# Perform one-hot encoding for categorical features
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(features)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [42]:
def get_daily_recommendation(home_or_gym, level, goal, bodyParts, equipments):
    if goal in ['Lose Weight', 'Get Fitter']:
      goal = "Get Fitter & Lose Weight"
    daily_recommendations = []

    bodyParts = [bp for bp in bodyParts if '-' not in bp]
     # Repeat elements in bodyParts until it reaches a size of 6
    while len(bodyParts) < 6:
        bodyParts += bodyParts

    # Limit bodyParts to size 6
    bodyParts = bodyParts[:6]

    for bodyPart in bodyParts:
        # Predict cluster for the specified combination of goal, level, and body part
        input_data = [[level, goal, bodyPart]]
        predicted_cluster = kmeans.predict(encoder.transform(input_data))[0]
        print(predicted_cluster)
        #Get data for the predicted cluster
        cluster_subset = cluster_data[predicted_cluster]

        # Filter data based on location (home or gym)
        if home_or_gym == 0:
            cluster_subset = cluster_subset[~cluster_subset["equipment"].isin(equipments)]

        # Randomly select one exercise from the cluster if any left after equipment filtering
        if not cluster_subset.empty:
            selected_exercise = random.choice(cluster_subset.to_dict(orient='records'))
            daily_recommendations.append(selected_exercise)

    # Remove duplicates from the list
    unique_recommendations = []
    seen_names = set()
    for exercise in daily_recommendations:
        if exercise['name'] not in seen_names:
            unique_recommendations.append(exercise)
            seen_names.add(exercise['name'])

    return unique_recommendations

# Example usage
home_or_gym = 0
level = "Beginner"
goal = "Get Fitter & Lose Weight"
bodyParts = ["waist", "back", "cardio"]
equipments = ["body weight", "dumbbell"]

recommendations = get_daily_recommendation(home_or_gym, level, goal, bodyParts, equipments)
print("Daily Recommendations:")
for idx, exercise in enumerate(recommendations, start=1):
    print(f"{idx}. {exercise}")


15
18
27
15
18
27
Daily Recommendations:
1. {'name': 'standing wheel rollerout', 'target': 'abs', 'equipment': 'wheel roller', 'Level': 'Beginner', 'gym': 1, 'home': 1, 'goal': 'Get Fitter & Lose Weight', 'bodyPart': 'waist', 'type': 'duration', 'sets': 4, 'cluster': 15}
2. {'name': 'reverse grip machine lat pulldown', 'target': 'lats', 'equipment': 'leverage machine', 'Level': 'Beginner', 'gym': 1, 'home': 0, 'goal': 'Get Fitter & Lose Weight', 'bodyPart': 'back', 'type': 'weight', 'sets': 4, 'cluster': 18}
3. {'name': 'cycle cross trainer', 'target': 'cardiovascular system', 'equipment': 'leverage machine', 'Level': 'Beginner', 'gym': 1, 'home': 0, 'goal': 'Get Fitter & Lose Weight', 'bodyPart': 'cardio', 'type': 'weight', 'sets': 4, 'cluster': 27}
4. {'name': 'stability ball crunch (full range hands behind head)', 'target': 'abs', 'equipment': 'Exercise Ball', 'Level': 'Beginner', 'gym': 1, 'home': 1, 'goal': 'Get Fitter & Lose Weight', 'bodyPart': 'waist', 'type': 'duration', 'sets

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHo

In [43]:
def get_gender_adjustment(gender):
    return 1.0 if gender == 'Male' else 0.7

def get_age_adjustment(age):
    if age < 30:
        return 1.0
    elif 30 <= age < 50:
        return 0.5
    else:
        return 0.1

def get_level_adjustment(level):
    if level == 'Beginner':
        return 0.8
    elif level == 'Intermediate':
        return 1.0
    elif level == 'Advanced':
        return 1.2

def get_body_part_adjustment(body_part):
    body_parts = {
        'chest': 1,
        'shoulders': 0.8,
        'waist': 0.6,
        'upper legs': 0.7,
        'back': 0.9,
        'lower legs': 0.5,
        'upper arms': 0.8,
        'cardio': 0.7,
        'lower arms': 0.6,
        'neck': 0.5
    }
    return body_parts.get(body_part, 0)
def adjust_workout(gender, age, feedback, body_part, level, old_weight):
    gender_adjustment = get_gender_adjustment(gender)
    age_adjustment = get_age_adjustment(age)
    level_adjustment = get_level_adjustment(level)
    body_part_adjustment = get_body_part_adjustment(body_part)

    increasing_factor_of_weight = age_adjustment * body_part_adjustment * gender_adjustment * level_adjustment * 0.3

    if not feedback:
        increasing_factor_of_weight = (1-increasing_factor_of_weight)*-0.1


    new_weight = old_weight + increasing_factor_of_weight * old_weight

    return  new_weight


gender = 'Female'
age = 20
feedback = False
body_part = 'chest'
level = 'Beginner'
old_weight = 20

new_weight = adjust_workout(gender, age, feedback, body_part, level, old_weight)

print("New weight:", new_weight)

New weight: 18.336


In [44]:
def calculate_new_repetition(level, goal):
    if goal in ['Lose Weight', 'Get Fitter']:
        if level == 'Beginner':
            return 15
        elif level == 'Intermediate':
            return 12
        elif level == 'Expert':
            return 10
    elif goal == 'Gain Muscle':
        if level == 'Beginner':
            return 10
        elif level == 'Intermediate':
            return 8
        elif level == 'Advanced':
            return 6

In [45]:
def calculate_new_duration(level):

    if level == 'Beginner':
        return 20
    elif level == 'Intermediate':
        return 50
    elif level == 'Advanced':
        return 80

In [46]:
def model(home_or_gym, level, goal, gender, age, feedback, old_weight, equipments):

    plan = choose_plan(level, goal, gender)
    print(plan)

    while len(plan) < 30:
        plan.extend(plan)
    plan = plan[:30]

    all_recommendations = []
    for day_body_parts in plan:
        daily_exercises = get_daily_recommendation(home_or_gym, level, goal, day_body_parts, equipments)
        daily_recommendations = []

        for exercise in daily_exercises:
            weights = adjust_workout(gender, age, feedback, exercise['bodyPart'], level, old_weight)
            repetitions = calculate_new_repetition(level, goal)
            duration = calculate_new_duration(level)
            weights_or_duration = weights if exercise['type'] == "weight" else duration
            exercise_recommendations = {
                'name': exercise['name'],
                'type': exercise['type'],
                'equipment': exercise['equipment'],
                'bodyPart': exercise['bodyPart'],
                'target': exercise['target'],
                'weights_or_duration': weights_or_duration,
                'sets': exercise['sets'],
                'repetitions': repetitions,
            }
            daily_recommendations.append(exercise_recommendations)
        all_recommendations.append(daily_recommendations)


    return all_recommendations  # Trim to ensure exactly 30 elements

In [47]:
class FitnessModel:
    def __init__(self, excercise_path, kmeans_path, plan_classifier_path):
        self.data = pd.read_csv(excercise_path)
        self.kmeans = None
        self.plan_classifier = None
        self.encoder = None
        self.cluster_data = {}
        self.X_train_cols = ['level_Advanced', 'level_Beginner', 'level_Intermediate',
                             'goal_ Get Fitter', 'goal_ Lose Weight', 'goal_Gain Muscle',
                             'goal_Get Fitter', 'goal_Increase Endurance', 'goal_Increase Strength',
                             'goal_Sports Performance', 'gender_Female', 'gender_Male',
                             'gender_Male & Female']

        # Load kmeans model
        with open(kmeans_path, 'rb') as f:
            self.kmeans = pickle.load(f)

        # Load plan classifier model
        with open(plan_classifier_path, 'rb') as f:
            self.plan_classifier = pickle.load(f)


        # Iterate over each cluster label
        for cluster_label in range(90):
            # Filter the dataset to get data for the current cluster
            cluster_subset = self.data[self.data["cluster"] == cluster_label]

            # Add the cluster data to the dictionary
            self.cluster_data[cluster_label] = cluster_subset

        features = self.data[["Level", "goal", "bodyPart"]]

        # Perform one-hot encoding for categorical features
        self.encoder = OneHotEncoder(sparse=False)
        encoded_features = self.encoder.fit_transform(features)

    def choose_plan(self,level, goal, gender):
        global plan_classifier
        # Convert input into a DataFrame
        input_data = pd.DataFrame({'level': [level], 'goal': [goal], 'gender': [gender]})

        # One-hot encode the input data
        input_encoded = pd.get_dummies(input_data, columns=['level', 'goal', 'gender'])

        # Ensure that input has the same columns as the model was trained on
        # This is necessary in case some categories are missing in the input
        missing_cols = set(self.X_train_cols) - set(input_encoded.columns)
        for col in missing_cols:
            input_encoded[col] = 0

        # Reorder columns to match the order of columns in X_train
        input_encoded = input_encoded[self.X_train_cols]

        # Make prediction for the given input using the trained model
        prediction = self.plan_classifier.predict(input_encoded)


        # Convert each string in the list to a list of strings
        daily_activities_lists = [day.split(', ') for day in prediction[0]]


        return daily_activities_lists



    def get_daily_recommendation(self, home_or_gym, level, goal, bodyParts, equipments):
      if goal in ['Lose Weight', 'Get Fitter']:
        goal = "Get Fitter & Lose Weight"
      daily_recommendations = []

      bodyParts = [bp for bp in bodyParts if '-' not in bp]
      # Repeat elements in bodyParts until it reaches a size of 6
      while len(bodyParts) < 6:
          bodyParts += bodyParts

      # Limit bodyParts to size 6
      bodyParts = bodyParts[:6]

      for bodyPart in bodyParts:
          # Predict cluster for the specified combination of goal, level, and body part
          input_data = [[level, goal, bodyPart]]
          predicted_cluster = self.kmeans.predict(self.encoder.transform(input_data))[0]
          print(predicted_cluster)
          #Get data for the predicted cluster
          cluster_subset = self.cluster_data[predicted_cluster]

          # Filter data based on location (home or gym)
          if home_or_gym == 0:
              cluster_subset = cluster_subset[~cluster_subset["equipment"].isin(equipments)]

          # Randomly select one exercise from the cluster if any left after equipment filtering
          if not cluster_subset.empty:
              selected_exercise = random.choice(cluster_subset.to_dict(orient='records'))
              daily_recommendations.append(selected_exercise)

      # Remove duplicates from the list
      unique_recommendations = []
      seen_names = set()
      for exercise in daily_recommendations:
          if exercise['name'] not in seen_names:
              unique_recommendations.append(exercise)
              seen_names.add(exercise['name'])

      return unique_recommendations

    def get_gender_adjustment(self,gender):
        return 1.0 if gender == 'Male' else 0.7

    def get_age_adjustment(self,age):
        if age < 30:
            return 1.0
        elif 30 <= age < 50:
            return 0.5
        else:
            return 0.1

    def get_level_adjustment(self,level):
        if level == 'Beginner':
            return 0.8
        elif level == 'Intermediate':
            return 1.0
        elif level == 'Advanced':
            return 1.2

    def get_body_part_adjustment(self,body_part):
        body_parts = {
            'chest': 1,
            'shoulders': 0.8,
            'waist': 0.6,
            'upper legs': 0.7,
            'back': 0.9,
            'lower legs': 0.5,
            'upper arms': 0.8,
            'cardio': 0.7,
            'lower arms': 0.6,
            'neck': 0.5
        }
        return body_parts.get(body_part, 0)
    def adjust_workout(self,gender, age, feedback, body_part, level, old_weight):
        gender_adjustment = self.get_gender_adjustment(gender)
        age_adjustment = self.get_age_adjustment(age)
        level_adjustment = self.get_level_adjustment(level)
        body_part_adjustment = self.get_body_part_adjustment(body_part)

        increasing_factor_of_weight = age_adjustment * body_part_adjustment * gender_adjustment * level_adjustment * 0.3

        if not feedback:
            increasing_factor_of_weight = (1-increasing_factor_of_weight)*-0.1


        new_weight = old_weight + increasing_factor_of_weight * old_weight

        return  new_weight

    def calculate_new_repetition(self,level, goal):
        if goal in ['Lose Weight', 'Get Fitter']:
            if level == 'Beginner':
                return 15
            elif level == 'Intermediate':
                return 12
            elif level == 'Expert':
                return 10
        elif goal == 'Gain Muscle':
            if level == 'Beginner':
                return 10
            elif level == 'Intermediate':
                return 8
            elif level == 'Advanced':
                return 6

    def calculate_new_duration(self,level):

        if level == 'Beginner':
            return 20
        elif level == 'Intermediate':
            return 50
        elif level == 'Advanced':
            return 80
    def predict(self,home_or_gym, level, goal, gender, age, feedback, old_weight, equipments):

      plan = self.choose_plan(level, goal, gender)
      print(plan)

      while len(plan) < 30:
          plan.extend(plan)
      plan = plan[:30]

      all_recommendations = []
      for day_body_parts in plan:
          daily_exercises = self.get_daily_recommendation(home_or_gym, level, goal, day_body_parts, equipments)
          daily_recommendations = []

          for exercise in daily_exercises:
              weights = self.adjust_workout(gender, age, feedback, exercise['bodyPart'], level, old_weight)
              repetitions = self.calculate_new_repetition(level, goal)
              duration = self.calculate_new_duration(level)
              weights_or_duration = weights if exercise['type'] == "weight" else duration
              exercise_recommendations = {
                  'name': exercise['name'],
                  'type': exercise['type'],
                  'equipment': exercise['equipment'],
                  'bodyPart': exercise['bodyPart'],
                  'target': exercise['target'],
                  'weights_or_duration': weights_or_duration,
                  'sets': exercise['sets'],
                  'repetitions': repetitions,
              }
              daily_recommendations.append(exercise_recommendations)
          all_recommendations.append(daily_recommendations)


      return all_recommendations  # Trim to ensure exactly 30 elements

In [48]:
recommendations = model(1, "Advanced", "Lose Weight", "Male", 22, True, 5, [])
for day_number, daily_recommendations in enumerate(recommendations, start=1):
    print("Day", day_number, "Recommendations:")
    for recommendation in daily_recommendations:
        print("Exercise Recommendations:")
        for key, value in recommendation.items():
            print(f"{key}: {value}")
        print()

[['upper legs', 'lower legs', 'back', 'back', 'back', 'shoulders', 'upper arms', 'upper arms'], ['chest', 'shoulders', 'chest', 'chest', 'shoulders', 'upper arms', 'upper arms'], ['upper legs', 'upper legs', 'lower legs', 'upper legs', 'upper legs', 'upper legs', 'lower arms', 'upper arms'], ['shoulders', 'chest', 'shoulders', 'chest', 'upper arms', 'upper arms', 'upper arms', 'upper arms'], ['upper legs', 'upper legs', 'lower legs', 'upper legs', 'upper legs', 'upper legs', 'lower legs', 'lower arms', 'upper arms'], ['back', 'back', 'back', 'back', 'upper arms', 'upper arms', 'upper arms', 'upper arms'], ['waist', 'waist', 'waist', 'waist', 'waist', 'waist']]
28
34
36
36
36
30
38
30
38
38
30
21
28
28
34
28
28
28
30
38
30
38
21
21
28
28
34
28
28
28
36
36
36
36
21
21
37
37
37
37
37
37
28
34
36
36
36
30
38
30
38
38
30
21
28
28
34
28
28
28
30
38
30
38
21
21
28
28
34
28
28
28


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHo

36
36
36
36
21
21
37
37
37
37
37
37
28
34
36
36
36
30
38
30
38
38
30
21
28
28
34
28
28
28
30
38
30
38
21
21
28
28
34
28
28
28
36
36
36
36
21
21
37
37
37
37
37
37
28
34
36
36
36
30
38
30
38
38
30
21
28
28
34
28
28
28
30
38
30
38
21


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHo

21
28
28
34
28
28
28
36
36
36
36
21
21
37
37
37
37
37
37
28
34
36
36
36
30
38
30
38
38
30
21
Day 1 Recommendations:
Exercise Recommendations:
name: sled 45 degrees one leg press
type: weight
equipment: sled machine
bodyPart: upper legs
target: glutes
weights_or_duration: 6.26
sets: 5
repetitions: None

Exercise Recommendations:
name: sled lying calf press
type: weight
equipment: sled machine
bodyPart: lower legs
target: calves
weights_or_duration: 5.9
sets: 4
repetitions: None

Exercise Recommendations:
name: smith shrug
type: weight
equipment: smith machine
bodyPart: back
target: traps
weights_or_duration: 6.62
sets: 5
repetitions: None

Exercise Recommendations:
name: weighted pull-up
type: weight
equipment: weighted
bodyPart: back
target: lats
weights_or_duration: 6.62
sets: 4
repetitions: None

Exercise Recommendations:
name: rope climb
type: weight
equipment: rope
bodyPart: back
target: upper back
weights_or_duration: 6.62
sets: 5
repetitions: None

Exercise Recommendations:
name:

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHo

type: weight
equipment: weighted
bodyPart: upper legs
target: glutes
weights_or_duration: 6.26
sets: 5
repetitions: None

Day 13 Recommendations:
Exercise Recommendations:
name: weighted one hand pull up
type: weight
equipment: weighted
bodyPart: back
target: lats
weights_or_duration: 6.62
sets: 4
repetitions: None

Exercise Recommendations:
name: weighted pull-up
type: weight
equipment: weighted
bodyPart: back
target: lats
weights_or_duration: 6.62
sets: 4
repetitions: None

Exercise Recommendations:
name: weighted muscle up
type: weight
equipment: weighted
bodyPart: back
target: lats
weights_or_duration: 6.62
sets: 3
repetitions: None

Exercise Recommendations:
name: dumbbell standing concentration curl
type: weight
equipment: dumbbell
bodyPart: upper arms
target: biceps
weights_or_duration: 6.4399999999999995
sets: 5
repetitions: None

Exercise Recommendations:
name: dumbbell standing one arm concentration curl
type: weight
equipment: dumbbell
bodyPart: upper arms
target: biceps
wei

In [49]:
exercise_path = '../data/final_datasets/excercise.csv'
kmeans_path = '../models/kmeans.pickle'
plan_classifier_path = '../models/plan_classifier.pickle'

# Create an instance of FitnessModel
fitness_model = FitnessModel(exercise_path, kmeans_path, plan_classifier_path)

# Print the instance to check its initialization
print(fitness_model)

# Save the instance using pickle
with open('fitness_model.pkl', 'wb') as file:
    pickle.dump(fitness_model, file)

print("FitnessModel instance saved successfully.")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


FitnessModel instance saved successfully.
